In [1]:
"""%pip install huggingface_hub
%pip install git+https://github.com/facebookresearch/segment-anything-2/
%pip install -r https://raw.githubusercontent.com/facebookresearch/segment-anything-2/main/requirements.txt"""
%pip install plotly



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pydicom as dicom
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from skimage.transform import resize
from skimage.io import imsave
from skimage import io 
from PIL import Image, ImageOps
from tqdm import tqdm
import os
import cv2
import math
import glob
import numpy as np
import albumentations as A
import imageio
import sys

import concurrent.futures

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sam2.sam2_image_predictor import SAM2ImagePredictor

import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import random
from glob import glob
import os, shutil
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd

# visualization
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm

# Albumentations for augmentations
from albumentations.pytorch import ToTensorV2

import rasterio
from joblib import Parallel, delayed

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Notebook Structure:
    Load in data
        - Load in csv files for default and 'improved' data
        - Combine the default and additional dataframes
        - Add filenames to the dataframes
    
    Preprocess imgs
        - Load in dicom stacks
        - Crop nessecary imgs
        - Save all as png

    Create Pytorch Dataset

    Create model
        - Download pretrained resnet18
        - Change the output size of the resnet18 model

    Train model
        - N epochs
        - Save the weights
    
    Preprocess Test Imgs


    Create Test Pytorch Dataset


    Use Model To Predict Severity
    
    

In [3]:
###################################################################################################################################
##################################################"""LOAD IN DATA"""###############################################################
###################################################################################################################################

In [4]:
train = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train.csv")
train_label_coordinates = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
train_series_descriptions = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")
test_series_description  = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv")
coords = pd.read_csv("coords_rsna_improved.csv")

train_labels = pd.merge(train_series_descriptions, train_label_coordinates, on = ['study_id', 'series_id'])

#Align formatting with the train.csv file as well as the sample submission
condition = train_labels['condition'].to_numpy()
reformatted_condition = np.array([cond.lower().replace(' ', '_') for cond in condition])
train_labels.iloc[::, 4] = reformatted_condition

level = train_labels['level'].to_numpy()
reformatted_level = np.array([l.lower().replace('/', '_') for l in level])
train_labels.iloc[::, 5] = reformatted_level

series_description = train_labels['series_description'].to_numpy()
reformatted_sd = np.array([d.replace(' ', '_').replace('/', '_') for d in series_description])
train_labels.iloc[::, 2] = reformatted_sd

def is_nan(value):
    return pd.isna(value)

def clean_nan_values(data):
    for index, row in data.iterrows():
        for i in range(len(row)):
            if is_nan(row[i]):
                row[i] = "Normal/Mild"
    return data

train = clean_nan_values(train)
train_labels

,study_id,series_id,series_description,instance_number,condition,level,x,y
0,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l1_l2,322.831858,227.964602
1,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l2_l3,320.571429,295.714286
2,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l3_l4,323.030303,371.818182
3,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l4_l5,335.292035,427.327434
4,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l5_s1,353.415929,483.964602
...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,Sagittal_T1,11,left_neural_foraminal_narrowing,l1_l2,219.465940,97.831063
48688,4290709089,4237840455,Sagittal_T1,12,left_neural_foraminal_narrowing,l2_l3,205.340599,140.207084
48689,4290709089,4237840455,Sagittal_T1,12,left_neural_foraminal_narrowing,l3_l4,202.724796,181.013624
48690,4290709089,4237840455,Sagittal_T1,12,left_neural_foraminal_narrowing,l4_l5,202.933333,219.733333


In [5]:
spinal_canal_columns = train.filter(like='spinal_canal').columns
train[spinal_canal_columns] = train[spinal_canal_columns].fillna('Normal/Mild')


def fill_nan_left_and_right(df, column):
    target_columns = df.filter(like=column).columns 
    null_rows = df[df[target_columns].isnull().any(axis=1)]
    for level in ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']:
        left_col = f"left_{column}_{level}"
        right_col = f"right_{column}_{level}"
        for index, row in null_rows.iterrows():
            if pd.isna(row[left_col]) and pd.isna(row[right_col]):
                df.at[index, left_col] = 'Normal/Mild'
                df.at[index, right_col] = 'Normal/Mild'
            
            elif pd.isna(row[right_col]):
                df.at[index, right_col] = row[left_col]
            
            elif pd.isna(row[left_col]):
                df.at[index, left_col] = row[right_col]
            else:
                continue
    return df
train = fill_nan_left_and_right(train, 'neural_foraminal_narrowing')
train = fill_nan_left_and_right(train, 'subarticular_stenosis')

In [6]:
coords = pd.read_csv('coords_rsna_improved.csv')

coords = coords.drop(['side'], axis = 1)
coords = coords.drop(coords.columns[0], axis=1)
coords['level'] = [level.lower().replace('/', '_') for level in coords['level'].to_list()]

coords = coords[['study_id', 'series_id', 'instance_number', 'condition', 'level', 'relative_x', 'relative_y']]
coords

,study_id,series_id,instance_number,condition,level,relative_x,relative_y
0,3996069892,10996,13,Left Neural Foraminal Narrowing,l1_l2,0.521148,0.325282
1,3996069892,10996,6,Right Neural Foraminal Narrowing,l1_l2,0.516856,0.319701
2,3996069892,10996,12,Left Neural Foraminal Narrowing,l2_l3,0.493773,0.400966
3,3996069892,10996,6,Right Neural Foraminal Narrowing,l2_l3,0.504032,0.385531
4,3996069892,10996,12,Left Neural Foraminal Narrowing,l3_l4,0.487331,0.475040
...,...,...,...,...,...,...,...
58730,916362094,4294540297,10,Spinal Canal Stenosis,l3_l4,0.511282,0.571751
58731,916362094,4294540297,-1,Spinal Canal Stenosis,l4_l5,0.308594,0.667969
58732,916362094,4294540297,9,Spinal Canal Stenosis,l4_l5,0.516697,0.671029
58733,916362094,4294540297,-1,Spinal Canal Stenosis,l5_s1,0.328125,0.796875


In [7]:
###################################################################################################################################
###############################################"""PREPROCESS DATA"""###############################################################
###################################################################################################################################

In [8]:
"""Translate Coordinates."""
def load_dcm_img(dcm_path) -> np.ndarray:
    dcm = dicom.dcmread(dcm_path)
    img: np.ndarray = dcm.pixel_array
    img = img.clip(np.percentile(img, 1), np.percentile(img, 99))
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    return img

def convert_to_8bit(x):
    lower, upper = np.percentile(x, (1, 99))
    x = np.clip(x, lower, upper)
    x = x - np.min(x)
    x = x / np.max(x)
    return (x * 255).astype("uint8")

def load_dicom_stack(dicom_folder, plane, reverse_sort=False):
    dicom_files = glob(os.path.join(dicom_folder, "*.dcm"))
    if not dicom_files:
        raise ValueError(f"No DICOM files found in {dicom_folder}")
        
    dicoms = [dicom.dcmread(f) for f in dicom_files]
    plane_index = {"sagittal": 0, "coronal": 1, "axial": 2}[plane.lower()]
    positions = np.asarray([float(d.ImagePositionPatient[plane_index]) for d in dicoms])

    idx = np.argsort(-positions if reverse_sort else positions)
    dicoms = [dicoms[i] for i in idx]
    max_shape = max([d.pixel_array.shape for d in dicoms])

    #Normalize the shape by padding with zeros or resizing
    normalized_arrays, padding_array = [], []
    for d in dicoms:
        img = d.pixel_array.astype("float32")
        if img.shape != max_shape:
            #Resize or pad the image to the max_shape
            padded_img = np.zeros(max_shape, dtype="float32")
            padded_img[:img.shape[0], :img.shape[1]] = img
            normalized_arrays.append(padded_img)

            x_padding = max_shape[1] - img.shape[1]
            y_padding = max_shape[0] - img.shape[0]
            padding_array.append((x_padding // 2, y_padding // 2))
        else:
            normalized_arrays.append(img)
            padding_array.append((0, 0))

    array = np.stack(normalized_arrays)
    ipp = np.asarray([d.ImagePositionPatient for d in dicoms]).astype("float")[idx]

    return {
        "array": convert_to_8bit(array), 
        "positions": ipp, 
        "pixel_spacing": np.asarray(dicoms[0].PixelSpacing).astype("float"),
        "padding_array": padding_array
    }

In [9]:
TRAIN_IMG_DIR = "rsna-2024-lumbar-spine-degenerative-classification/train_images"
normal_tl = "normal_train_labels.csv"

if os.path.exists(normal_tl) == False:
    for study_folder_path in tqdm(os.listdir(TRAIN_IMG_DIR)):
        series_folders = os.listdir(os.path.join(TRAIN_IMG_DIR, study_folder_path))
        for idx, series_folder in enumerate(series_folders):
            dcm_folder_path = os.path.join(TRAIN_IMG_DIR, study_folder_path, series_folder)

            df = train_labels[train_labels["series_id"] == int(series_folder)]
            try:
                series_desc = df["series_description"].values[0]
            except:
                print(f"Series description for series_id: {series_folder} not found! Skipping.")
                continue

            plane = "sagittal" if series_desc != "Axial T2" else "axial"
            dicom_data_3d = load_dicom_stack(dcm_folder_path, plane = plane)
            padding_array = dicom_data_3d["padding_array"]
            
            for i in range(len(df["x"].to_list())):
                x_pad, y_pad = padding_array[i]
                df.iloc[i, df.columns.get_loc("x")] = df.iloc[i, df.columns.get_loc("x")] + x_pad
                df.iloc[i, df.columns.get_loc("y")] = df.iloc[i, df.columns.get_loc("y")] + y_pad


            pixel_spacing = dicom_data_3d["pixel_spacing"]
            img = dicom_data_3d["array"][idx]
            img_shape = img.shape

            x_scale_factor = 224 / img_shape[1]
            y_scale_factor = 224 / img_shape[0]

            train_labels.loc[df.index, "x"] = df["x"] * (x_scale_factor)
            train_labels.loc[df.index, "y"] = df["y"] * (y_scale_factor)
        

In [10]:
"""Verify Coordinates are between 0 and 224."""
print(len(train_labels))
tl_in_lims = train_labels[(train_labels["x"] >= 0) & (train_labels["x"] <= 224) & (train_labels["y"] >= 0) & (train_labels["y"] <= 224)]
print(len(tl_in_lims))

48692
18114


In [11]:
print(len(train_labels))
print(train_labels.head(10))

48692
   study_id   series_id series_description  instance_number  \
0   4003253   702807833   Sagittal_T2_STIR                8   
1   4003253   702807833   Sagittal_T2_STIR                8   
2   4003253   702807833   Sagittal_T2_STIR                8   
3   4003253   702807833   Sagittal_T2_STIR                8   
4   4003253   702807833   Sagittal_T2_STIR                8   
5   4003253  1054713880        Sagittal_T1                4   
6   4003253  1054713880        Sagittal_T1                4   
7   4003253  1054713880        Sagittal_T1                5   
8   4003253  1054713880        Sagittal_T1                6   
9   4003253  1054713880        Sagittal_T1                6   

                          condition  level           x           y  
0             spinal_canal_stenosis  l1_l2  322.831858  227.964602  
1             spinal_canal_stenosis  l2_l3  320.571429  295.714286  
2             spinal_canal_stenosis  l3_l4  323.030303  371.818182  
3             spinal_can

In [12]:
if os.path.exists(normal_tl):
    train_labels = pd.read_csv(normal_tl)
else:
    train_labels.to_csv(normal_tl, index=False)

In [13]:
def resample_img(img, pixel_spacing, new_spacing):
    current_spacing = np.array(pixel_spacing, dtype=np.float32)
    resize_factor = current_spacing / new_spacing
    new_shape = np.round(img.shape * resize_factor).astype(int)
    resampled_image = resize(img, new_shape, preserve_range=True, anti_aliasing=True)
    return resampled_image

def resize_img(img):
    return resize(img, (224, 224), preserve_range=True, anti_aliasing=True)

#def reduce_noise(img):
    #return cv2.fastNlMeansDenoising(img, None, 5, 7, 21)

def normalize_img(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img))

def preprocess_img(img, pixel_spacing):
    resampled_img = resample_img(img, pixel_spacing, (1.0, 1.0))
    resized_img = resize_img(resampled_img)
    #denoised_img = reduce_noise(resized_img)
    normalized_img = normalize_img(resized_img)
    return normalized_img

In [14]:
"""Functions which get the dicom info and use it to create a preprocessed img, whilst also scaling the coords correctly"""
def save_img(img, filename, save_folder):
    os.makedirs(save_folder, exist_ok = True)
    imageio.imwrite(os.path.join(save_folder, filename), (img * 255).astype(np.uint8))

In [15]:
if os.path.exists("train_png") == False:
    for study_folder_path in tqdm(os.listdir(TRAIN_IMG_DIR)):
        series_folders = os.listdir(os.path.join(TRAIN_IMG_DIR, study_folder_path))
        for series_folder in series_folders:
            dcm_folder_path = os.path.join(TRAIN_IMG_DIR, study_folder_path, series_folder)
            df = train_series_descriptions[train_series_descriptions["series_id"] == int(series_folder)]
            try:
                series_desc = df["series_description"].values[0][0]
            except:
                print(f"Series description for series_id: {series_folder} not found! Skipping.")
                continue

            plane = "sagittal" if series_desc != "Axial T2" else "axial"
            dicom_data_3d = load_dicom_stack(dcm_folder_path, plane = plane)
            
            if dicom_data_3d["array"].shape[0] == 0:
                print(f"Can't loop over size zero dicom data: {dicom_data_3d["array"].shape}")
                continue
            
            for k in range(dicom_data_3d["array"].shape[0]):
                img = dicom_data_3d["array"][k]
                pixel_spacing = dicom_data_3d["pixel_spacing"]
                processed_img = preprocess_img(img, pixel_spacing)
                save_folder = f"train_png/{study_folder_path}/{series_folder}"
                filename = f"{k+1}.png"
                save_img(processed_img, filename=filename, save_folder=save_folder)
           

In [16]:
d = {"study_id": "first", "series_description": "first", "condition": "first", "level": list, "x": list, "y": list}
train_labels_grouped = train_labels.groupby(["series_id", "instance_number"], as_index=False).aggregate(d).reindex(columns=train_labels.columns)

In [17]:
def read_img(img_file):
    img_file = img_file.strip()
    #Get img_file path, issue of a '/x' on the end.
    if img_file[-1] == 'x':
        img_file = img_file.removesuffix('/x')
    return Image.open(img_file)

In [18]:
IMG_PATHS = glob(os.path.join("train_png", "*/*/*png"))

ST1_IMG_PATHS, ST2_IMG_PATHS, AX_IMG_PATHS = [], [], []
st1_train_df = train_series_descriptions[(train_series_descriptions["series_description"] == "Sagittal_T1") | (train_series_descriptions["series_description"] == "Sagittal T1")]
st1_sids = st1_train_df["series_id"].to_list()
st2_train_df = train_series_descriptions[(train_series_descriptions["series_description"] == "Sagittal_T2_STIR") | (train_series_descriptions["series_description"] == "Sagittal T2/STIR")]
st2_sids = st2_train_df["series_id"].to_list()
ax_train_df = train_series_descriptions[(train_series_descriptions["series_description"] == "Axial_T2") | (train_series_descriptions["series_description"] == "Axial T2")]
ax_sids = ax_train_df["series_id"].to_list()

for path in tqdm(IMG_PATHS, maxinterval=len(IMG_PATHS)):
    if any(str(sid) in path for sid in st1_sids):
        ST1_IMG_PATHS.append(path)
    elif any(str(sid) in path for sid in st2_sids):
        ST2_IMG_PATHS.append(path)
    elif any(str(sid) in path for sid in ax_sids):
        AX_IMG_PATHS.append(path)

print(len(IMG_PATHS), len(ST1_IMG_PATHS), len(ST2_IMG_PATHS), len(AX_IMG_PATHS))

100%|██████████| 147218/147218 [00:41<00:00, 3572.67it/s]

147218 33670 33569 79979


In [19]:
###################################################################################################################################
############################################ SEGMENT IMAGES USING UNET ############################################################
##########################################INSERT KAGGLE USERNAME AND LINK##########################################################

In [20]:
"""Using a UNet kaggle notebook, segment the discs."""
class CFG:
    seed          = 101
    debug         = False #Set debug = False for Full Training.
    exp_name      = 'Baselinev2'
    comment       = 'unet-efficientnet_b1-224x224-aug2-split2'
    model_name    = 'Unet'
    backbone      = 'efficientnet-b1'
    train_bs      = 32          #Reduced on the mac from 128 due to memory failure of mps.
    valid_bs      = train_bs    #Usually train_bs * 2, but changed due to memory failure.
    img_size      = [224, 224]
    epochs        = 25
    lr            = 2e-3
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = max(1, 32//train_bs)
    n_fold        = 5
    num_classes   = 3
    device        = torch.device("mps" if torch.backends.mps.is_available() else "cpu") #CHANGE ON KAGGLE FOR CUDA

df = pd.read_csv("dataset.csv")

def id2mask(id_):
    idf = df[df['id']==id_]
    wh = idf[['height','width']].iloc[0]
    shape = (wh.height, wh.width, 3)
    mask = np.zeros(shape, dtype=np.uint8)
    for i, class_ in enumerate(['large_bowel', 'small_bowel', 'stomach']):
        cdf = idf[idf['class']==class_]
        rle = cdf.segmentation.squeeze()
        if len(cdf) and not pd.isna(rle):
            mask[..., i] = rle_decode(rle, shape[:2])
    return mask

def rgb2gray(mask):
    pad_mask = np.pad(mask, pad_width=[(0,0),(0,0),(1,0)])
    gray_mask = pad_mask.argmax(-1)
    return gray_mask

def gray2rgb(mask):
    rgb_mask = tf.keras.utils.to_categorical(mask, num_classes=4)
    return rgb_mask[..., 1:].astype(mask.dtype)

def load_img(path):
    if path[0] == '/':
        path = path[1:]
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    img = np.tile(img[...,None], [1, 1, 3]) # gray to rgb
    img = img.astype('float32') # original is uint16
    mx = np.max(img)
    if mx:
        img/=mx # scale image to [0, 1]
    return img

def load_msk(path):
    msk = np.load(path, allow_pickle=True)
    msk = msk.astype('float32')
    msk/=255.0
    return msk
    
def show_img(img, mask=None):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    plt.imshow(img, cmap='bone')
    
    if mask is not None:
        plt.imshow(mask, alpha=0.5)
        handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]

    plt.axis('off')

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

skf = StratifiedGroupKFold(n_splits=2, shuffle=True, random_state=CFG.seed)
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['empty'], groups = df["case"])):
    df.loc[val_idx, 'fold'] = fold
display(df.groupby(['fold','empty'])['id'].count())

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=True, transforms=None):
        self.df         = df
        self.label      = label
        self.img_paths  = df['image_slices (png)'].tolist()
        self.msk_paths  = df['mask_slices (png)'].tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = self.img_paths[index]
        img = []
        img = load_img(img_path)
        
        if self.label:
            msk_path = self.msk_paths[index]
            msk = load_img(msk_path)
            if self.transforms:
                data = self.transforms(image=img, mask=msk)
                img  = data['image']
                msk  = data['mask']
            img = np.transpose(img, (2, 0, 1))
            msk = np.transpose(msk, (2, 0, 1))
            return torch.tensor(img), torch.tensor(msk)
        else:
            if self.transforms:
                data = self.transforms(image=img)
                img  = data['image']
            img = np.transpose(img, (2, 0, 1))
            return torch.tensor(img)

data_transforms = {
    "train": A.Compose([
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
# #             A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=1.0)
        ], p=0.25),
        A.CoarseDropout(max_holes=8, max_height=CFG.img_size[0]//20, max_width=CFG.img_size[1]//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        ], p=1.0),
    
    "valid": A.Compose([
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        ], p=1.0)
}

def prepare_loaders(fold, debug=False):
    train_df = df.query("fold!=@fold").reset_index(drop=True)
    valid_df = df.query("fold==@fold").reset_index(drop=True)
    if debug:
        train_df = train_df.head(32*5).query("empty==0")
        valid_df = valid_df.head(32*3).query("empty==0")
    train_dataset = BuildDataset(train_df, transforms=data_transforms['train'])
    valid_dataset = BuildDataset(valid_df, transforms=data_transforms['valid'])

    train_loader = DataLoader(train_dataset, batch_size=CFG.train_bs if not debug else 20, 
                              num_workers=0, shuffle=True, pin_memory=True, drop_last=False)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.valid_bs if not debug else 20, 
                              num_workers=0, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

train_loader, valid_loader= prepare_loaders(fold=0, debug=True)
imgs, msks = next(iter(train_loader))
imgs.size(), msks.size()

import gc
gc.collect()

import segmentation_models_pytorch as smp

def build_model():
    model = smp.Unet(
        encoder_name=CFG.backbone,      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=CFG.num_classes,        # model output channels (number of classes in your dataset)
        activation=None,
    )
    #model.encoder.load_state_dict(torch.load('/kaggle/input/efficientnet-b1-1c1461ff.pth'))
    model.to(CFG.device)
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load("best_epoch-00.bin"))
    model.eval()
    return model

JaccardLoss = smp.losses.JaccardLoss(mode='multilabel')
DiceLoss    = smp.losses.DiceLoss(mode='multilabel')
BCELoss     = smp.losses.SoftBCEWithLogitsLoss()
LovaszLoss  = smp.losses.LovaszLoss(mode='multilabel', per_image=False)
TverskyLoss = smp.losses.TverskyLoss(mode='multilabel', log_loss=False)

def dice_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean(dim=(1,0))
    return dice

def iou_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    union = (y_true + y_pred - y_true*y_pred).sum(dim=dim)
    iou = ((inter+epsilon)/(union+epsilon)).mean(dim=(1,0))
    return iou

def criterion(y_pred, y_true):
    return 0.5*BCELoss(y_pred, y_true) + 0.5*TverskyLoss(y_pred, y_true)

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = criterion(y_pred, masks)
            loss   = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (images, masks) in pbar:        
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        loss    = criterion(y_pred, masks)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        y_pred = nn.Sigmoid()(y_pred)
        val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        val_scores.append([val_dice, val_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    val_scores  = np.mean(val_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, val_scores

def run_training(model, optimizer, scheduler, device, num_epochs):    
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_dice      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CFG.device, epoch=epoch)
        
        val_loss, val_scores = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch)
        val_dice, val_jaccard = val_scores
    
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Dice'].append(val_dice)
        history['Valid Jaccard'].append(val_jaccard)
        
        # Log the metrics
        metrics = {"Train Loss": train_loss, 
                    "Valid Loss": val_loss,
                    "Valid Dice": val_dice,
                    "Valid Jaccard": val_jaccard,
                    "LR":scheduler.get_last_lr()[0]}
        
        print(f'Valid Dice: {val_dice:0.4f} | Valid Jaccard: {val_jaccard:0.4f}')

        if val_dice >= best_dice:
            print(f"{c_}Valid Score Improved ({best_dice:0.4f} ---> {val_dice:0.4f})")
            best_dice    = val_dice
            best_jaccard = val_jaccard
            best_epoch   = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            print(f"Model Saved{sr_}")

        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
                
    end = time.time()
    time_elapsed = end - start
    
    model.load_state_dict(best_model_wts)
    return model, history

def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == None:
        return None
        
    return scheduler

if not os.path.exists("last_epoch-00.bin"):
    model = build_model()
    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler = fetch_scheduler(optimizer)
    for fold in range(1):
        print(f'#' * 15)
        print(f'### Fold: {fold}')
        print(f'#' * 15)

        # Prepare loaders and model
        train_loader, valid_loader = prepare_loaders(fold=fold, debug=CFG.debug)
        model = build_model()
        optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
        scheduler = fetch_scheduler(optimizer)

        # Print basic info instead of wandb logging
        print(f"Starting training for fold-{fold} with model {CFG.model_name} and dimensions {CFG.img_size[0]}x{CFG.img_size[1]}")

        # Training loop
        model, history = run_training(model, optimizer, scheduler,
                                    device=CFG.device,
                                    num_epochs=CFG.epochs)

        # Optionally, print completion message
        print(f"Completed fold-{fold}")

def plot_batch(imgs, msks, size=3):
    plt.figure(figsize=(5*5, 5))
    for idx in range(size):
        plt.subplot(1, 5, idx+1)
        img = imgs[idx,].permute((1, 2, 0)).numpy()*255.0
        img = img.astype('uint8')
        msk = msks[idx,].permute((1, 2, 0)).numpy()*255.0
        show_img(img, msk)
    plt.tight_layout()
    plt.show()


fold  empty
0.0   False    341
      True     127
1.0   False    297
      True     132
Name: id, dtype: int64

In [21]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    device = torch.device("mps")


In [22]:
import matplotlib.image
from lumbar_dataset import *

model = load_model(f"best_epoch-00.bin").to(device)
model.eval()

st2_predict_dataset = RSNATrainPredict(ST2_IMG_PATHS)
print(len(st2_predict_dataset))
segmentation_dataloader = DataLoader(st2_predict_dataset, batch_size=64,
                          num_workers=8, shuffle=False, pin_memory=True)

def save_prediction(img_path, pred):
    filename = img_path.replace('train_png', 'nnUNet_segments').replace('png', 'npy')
    folder_path = "/".join(filename.split('/')[:-1])
    os.makedirs(folder_path, exist_ok=True)
    gray_mask = pred[:, :, 0]
    np.save(filename, gray_mask)

def process_and_save(data):
    img, img_paths = data["img"].to(device), data["img_path"]
    with torch.no_grad():
        pred = model(img)
        pred = (nn.Sigmoid()(pred) > 0.5).float()
    pred = pred.detach().cpu().numpy()
    pred = np.transpose(pred, (0, 2, 3, 1)) 
    for k in range(pred.shape[0]): #Batch_size.
        current_pred, current_instance = pred[k], img_paths[k].split('/')[-1].split('.')[0]
        if int(current_instance) <= 3 and np.count_nonzero(current_pred == 1.) > 50: #Need to mess around with it to find the best miniumum number of pixels.
            save_prediction(img_paths[k], current_pred) 
        elif int(current_instance) <= 3 and np.count_nonzero(current_pred == 1.) > 50: #Again, do testing.
            save_prediction(img_paths[k], current_pred) 
        elif np.count_nonzero(current_pred == 1.) > 50:
            save_prediction(img_paths[k], current_pred) 

if not os.path.exists("nnUNet_segments"):
    for data in tqdm(segmentation_dataloader, total=int(len(st2_predict_dataset) / 64)):
        process_and_save(data)


33569


In [23]:
st1_predict_dataset = RSNATrainPredict(ST1_IMG_PATHS)
print(len(st1_predict_dataset))
segmentation_dataloader = DataLoader(st1_predict_dataset, batch_size=64,
                          num_workers=8, shuffle=False, pin_memory=True)

if not os.path.exists("nnUNet_segments"):
    for data in tqdm(segmentation_dataloader, total=int(len(st1_predict_dataset) / 64)):
        process_and_save(data)

print(len(glob(os.path.join("nnUNet_segments", "*/*/*npy"))))

33670
36728


In [24]:
#################################################################################################################################
############################################# SEGMENT SPINAL CANAL WITH SAM2 ####################################################
#################################################################################################################################

In [25]:
"""Now using Meta's SAM2 model, obtain the spinal canal or degeneracies in the imgs.
from process_imgs import *
from multiprocessing import Pool

predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-large", device="cpu")

#CREATE A FOLDER OF SUITABLE SEGEMENTATIONS OF 

df = train_labels.copy()
df = df.set_index(["study_id", "series_id", "instance_number"])[["x", "y", "condition"]]
df = df.drop([df["condition"].str.contains("subarticular")].index) 
df = df.drop([df["condition"].str.contains("foraminal")].index) 

o = 0
for (study_id, series_id, instance_number), chunk in tqdm(df.groupby(level=[0, 1, 2])[["x", "y"]]):
    process_image(study_id, series_id, instance_number, chunk)
    o+=1
    if o == 2: break

if not os.path.exists("nnUNet_spinal_canal"):
    with Pool(processes=10) as pool:
        args = [(study_id, series_id, instance_number, chunk) for (study_id, series_id, instance_number), chunk in df.groupby(level=[0, 1, 2])[["x", "y"]]]
        tqdm(pool.starmap(process_image, args), maxinterval = len(df.groupby(level=[0, 1, 2])[["x", "y"]]))"""

'Now using Meta\'s SAM2 model, obtain the spinal canal or degeneracies in the imgs.\nfrom process_imgs import *\nfrom multiprocessing import Pool\n\npredictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-large", device="cpu")\n\n#CREATE A FOLDER OF SUITABLE SEGEMENTATIONS OF \n\ndf = train_labels.copy()\ndf = df.set_index(["study_id", "series_id", "instance_number"])[["x", "y", "condition"]]\ndf = df.drop([df["condition"].str.contains("subarticular")].index) \ndf = df.drop([df["condition"].str.contains("foraminal")].index) \n\no = 0\nfor (study_id, series_id, instance_number), chunk in tqdm(df.groupby(level=[0, 1, 2])[["x", "y"]]):\n    process_image(study_id, series_id, instance_number, chunk)\n    o+=1\n    if o == 2: break\n\nif not os.path.exists("nnUNet_spinal_canal"):\n    with Pool(processes=10) as pool:\n        args = [(study_id, series_id, instance_number, chunk) for (study_id, series_id, instance_number), chunk in df.groupby(level=[0, 1, 2])[["x", "y"]]]\n      

Classes in the segmentation mask.
0: Background/unimportant
1: Spinal canal
2: L1_L2 disc
...
6: L5_S1 disc
7: Normal/mild degeneration
8: Moderate degeneration
9: Severe degeneration
10: Other Discs.

In [26]:
###################################################################################################################################
################################################### NNUNET SETUP ##################################################################
###################################################################################################################################

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

nnUNet_raw/
├── Dataset_LumbarDegeneration
    ├── dataset.json  #Containing the metadata of the dataset.
    ├── imagesTr
        │   ├── 702807833_1_0000.png  #seriesID_instanceNumber_000(0,1,2).png.
        │   ├── 1054713880_1_0001.png  #0000: Sagittal T1, 0001: Sagittal T2/STIR, 0002: Axial T2
        │   ├── 2448190387_1_0002.png
    ├── imagesTs  #Test images (optional).
    └── labelsTr
        ├── 702807833_1.png
        ├── 1054713880_1.png
        ├── ...

In [27]:
UNET_ST1_DIR = "nnUNet_raw/Dataset001_ST1_Degeneration"
UNET_ST2_DIR = "nnUNet_raw/Dataset002_ST2_Degeneration"
os.makedirs(os.path.join(UNET_ST1_DIR, "imagesTr"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST1_DIR, "imagesTs"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST1_DIR, "labelsTr"), exist_ok = True)

os.makedirs(os.path.join(UNET_ST2_DIR, "imagesTr"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST2_DIR, "imagesTs"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST2_DIR, "labelsTr"), exist_ok = True)

os.makedirs("nnUNet_preprocessed", exist_ok = True)
os.makedirs("nnUNet_results", exist_ok = True)

In [28]:
"""Create dataset.json"""
import json
s1,s2 = "sagittal_t1", "sagittal_t2"
lf,rf,scs = "left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing", "spinal_canal_stenosis"
l1,l2,l3,l4,l5 = "l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"
n,m,s,i = "normal_mild", "moderate", "severe", "intact"

st1_classes = {
        "background": 0,
        "other_disc": 1,

        f"{lf}_{l1}_{n}": 2, #Sagittal T1: left_neural_foraminal_narrowing n/m/s.
        f"{lf}_{l2}_{n}": 3,
        f"{lf}_{l3}_{n}": 4,
        f"{lf}_{l4}_{n}": 5,
        f"{lf}_{l5}_{n}": 6,
        f"{lf}_{l1}_{m}": 7,
        f"{lf}_{l2}_{m}": 8,
        f"{lf}_{l3}_{m}": 9,
        f"{lf}_{l4}_{m}": 10,
        f"{lf}_{l5}_{m}": 11,
        f"{lf}_{l1}_{s}": 12,
        f"{lf}_{l2}_{s}": 13,
        f"{lf}_{l3}_{s}": 14,
        f"{lf}_{l4}_{s}": 15,
        f"{lf}_{l5}_{s}": 16,

        f"{rf}_{l1}_{n}": 17, #Sagittal T1: right_neural_foraminal_narrowing n/m/s.
        f"{rf}_{l2}_{n}": 18,
        f"{rf}_{l3}_{n}": 19,
        f"{rf}_{l4}_{n}": 20,
        f"{rf}_{l5}_{n}": 21,
        f"{rf}_{l1}_{m}": 22,
        f"{rf}_{l2}_{m}": 23,
        f"{rf}_{l3}_{m}": 24,
        f"{rf}_{l4}_{m}": 25,
        f"{rf}_{l5}_{m}": 26,
        f"{rf}_{l1}_{s}": 27,
        f"{rf}_{l2}_{s}": 28,
        f"{rf}_{l3}_{s}": 29,
        f"{rf}_{l4}_{s}": 30,
        f"{rf}_{l5}_{s}": 31,

        f"{s1}_l1": 32, #Sagittal T1 disc.
        f"{s1}_l2": 33,
        f"{s1}_l3": 34,
        f"{s1}_l4": 35,
        f"{s1}_l5": 36,
        f"{s1}_s1": 37,
    }

st2_classes = {
        "background": 0,
        "other_disc": 1,

        f"{scs}_{l1}_{n}": 2, #Sagittal T2/STIR spinal_canal_stenosis n/m/s.
        f"{scs}_{l2}_{n}": 3,
        f"{scs}_{l3}_{n}": 4,
        f"{scs}_{l4}_{n}": 5,
        f"{scs}_{l5}_{n}": 6,
        f"{scs}_{l1}_{m}": 7,
        f"{scs}_{l2}_{m}": 8,
        f"{scs}_{l3}_{m}": 9,
        f"{scs}_{l4}_{m}": 10,
        f"{scs}_{l5}_{m}": 11,
        f"{scs}_{l1}_{s}": 12,
        f"{scs}_{l2}_{s}": 13,
        f"{scs}_{l3}_{s}": 14,
        f"{scs}_{l4}_{s}": 15,
        f"{scs}_{l5}_{s}": 16, 

        f"{s2}_l1": 17, #Sagittal T2/STIR disc.
        f"{s2}_l2": 18,
        f"{s2}_l3": 19,
        f"{s2}_l4": 20,
        f"{s2}_l5": 21,
        f"{s2}_s1": 22,
}

In [29]:
"""Add 'inferenced' degenerative rows to the train_labels dataframe."""
def inference_rows(rows, instance):
    rows = rows.drop_duplicates(subset = ["level"])
    rows["instance_number"] = instance
    return rows

erows = pd.DataFrame(columns = train_labels.columns)
sagittal_idf = train_labels[(train_labels["condition"] != "left_subarticular_stenosis") | (train_labels["condition"] != "right_subarticular_stenosis")]
print(len(sagittal_idf))

for idx, chunk in sagittal_idf.groupby("series_id"):
    chunk = chunk.reset_index()
    if len(chunk) < 2:
        continue #Skip any with 1 row in as no data

    for instance in np.unique(chunk["instance_number"].to_numpy()):
        valid_ins = [instance - 1, instance, instance + 1]
        valid_cond = chunk[chunk["instance_number"] == instance]["condition"].values[0]
        valid_rows = chunk[(chunk["instance_number"].isin(valid_ins)) & (chunk["condition"] == valid_cond)]
        irows = inference_rows(valid_rows, instance)
        erows = pd.concat([erows, irows], ignore_index = True)

print(len(sagittal_idf), len(erows))
itrain_labels = pd.concat([sagittal_idf, erows], ignore_index = True)
print(len(itrain_labels))
itrain_labels = itrain_labels.drop_duplicates(subset=['series_id', 'instance_number', 'level'])
itrain_labels = itrain_labels[train_labels.columns]
print(len(itrain_labels))

48692
48692 61899
110591
61941


In [30]:
itrain_labels = itrain_labels.sort_values(['series_id', 'instance_number']).reset_index()
itrain_labels = itrain_labels[train_labels.columns]
print(itrain_labels[(itrain_labels["series_id"] == 1054713880) & (itrain_labels["instance_number"] == 11)])

      study_id   series_id series_description instance_number  \
15243  4003253  1054713880        Sagittal_T1              11   
15244  4003253  1054713880        Sagittal_T1              11   
15245  4003253  1054713880        Sagittal_T1              11   
15246  4003253  1054713880        Sagittal_T1              11   
15247  4003253  1054713880        Sagittal_T1              11   

                             condition  level           x           y  
15243  left_neural_foraminal_narrowing  l1_l2  114.374558   73.512367  
15244  left_neural_foraminal_narrowing  l4_l5  108.794275  146.762075  
15245  left_neural_foraminal_narrowing  l5_s1  114.975332  168.850095  
15246  left_neural_foraminal_narrowing  l2_l3  111.604240   99.236749  
15247  left_neural_foraminal_narrowing  l3_l4  109.595707  126.726297  


In [31]:
"""Create appropriate dataframe containing the severity of the coordinate for the classification of each disc in each image"""
class_df = pd.merge(itrain_labels, train_series_descriptions, on = ["study_id", "series_id"])
class_df = class_df[class_df["series_description_x"] != "Axial_T2"].drop("series_description_y", axis = 1).rename(columns={"series_description_x": "series_description"})
class_df = class_df.dropna()

def rename_cell(col, val):
    return f"{col}_{val.lower().replace('/', '_')}"

def get_class(row):
    col_name = f"{row['condition']}_{row['level']}"  
    #print(col_name)
    if col_name in row.index: 
        return row[col_name]
    else:
        print(f"No column with name: {col_name}\n{row}"); return None

def class_to_number(cell, classes):
    return classes[cell] if classes[cell] is not None else np.nan

def process_nnunet_df(class_df, classes, st1_st2):
    train_copy = train[train.columns[:16]] #Drop subarticular columns. (Axial T2).
    n = 1 if st1_st2 == "st2" else 6
    m = 6 if st1_st2 == "st2" else 16
    cols = [train.columns[0]] + list(train.columns[n:m])
    train_copy = train[cols] #Drop correct condition columns, depending on sagt1 or t2. Including the study_id column.
    for col in train_copy.columns[1:]:
        train_copy[col] = train_copy[col].apply(lambda x: rename_cell(col, x))

    class_df = pd.merge(class_df, train_copy, on = "study_id")
    class_df['class_number'] = class_df.apply(get_class, axis=1)
    class_df = class_df.drop(class_df.columns[8:-1], axis = 1)
    class_df["class_number"] = class_df["class_number"].apply(lambda x: class_to_number(x, classes))
    if len(class_df.dropna()) != len(class_df): print("nans in the dataframe") 
    return class_df

class_df = class_df.drop(
    class_df[
        (class_df["series_description"] == "Sagittal_T1") & 
        (class_df["condition"].str.contains('canal'))
    ].index
)

class_df = class_df.drop(
    class_df[
        (class_df["series_description"] == "Sagittal_T2_STIR") & 
        (class_df["condition"].str.contains('narrowing'))
    ].index
)

st1_class_df = process_nnunet_df(class_df[class_df["series_description"] == "Sagittal_T1"], st1_classes, "st1")
st1_class_df = st1_class_df.dropna()
print("unqiue elements:", np.unique(class_df["series_description"].to_numpy()))
st2_class_df = process_nnunet_df(class_df[class_df["series_description"] == "Sagittal_T2_STIR"], st2_classes, "st2")
st2_class_df = st2_class_df.dropna()

if np.unique(st1_class_df["class_number"].to_numpy()).shape[0] != 30:
    print("Error in st1_class_df") 
if np.unique(st2_class_df["class_number"].to_numpy()).shape[0] != 15:
    print("Error in st2_class_df") 

unqiue elements: ['Sagittal_T1' 'Sagittal_T2_STIR']


In [32]:
from scipy.ndimage import label
from skimage.measure import regionprops


"""Label the masks with the vertebrae class as well as the disc class. (From either st1_classes or st2_classes)"""
"""General Process: -> Take the highest disc from the dataframe (l1_l2)
-> Find the center and sides of the vertebrae
-> Fill a rectangle below the disc, not overlapping the vertebrae below, representing the disc between the two vertebrae
-> Append to an array when completed
-> Repeat until l5_s1 is reached
-> Label s1
-> Compare the array of completed discs/vertebrae to the list of all discs
-> Move up to staring vertebrae, until all discs/vertebrae are completed
-> Label the remaining vertebrae as "other_disc": 2 #The name for these is wrong, however is it unimportant for now.
"""

def auto_label_mask(mask):
    labeled_mask, num_features = label(mask) 
    return labeled_mask
    
def correct_labels_for_overlap(mask, classes):
    #First change the auto-assigned background color(s) to 0.
    unique, counts = np.unique(mask, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1] #Sort in descending order
    class_values = classes.values()
    for u in unique:
        if u == 0:
            continue
        if u in class_values:
            mask[mask == u] = u + 70 #70 ensures the overlapping pixel value is now completely unique.
    return mask

def auto_label_and_correct_mask(mask, classes):
    color_mask = auto_label_mask(mask)
    corrected_color_mask = correct_labels_for_overlap(color_mask, classes)
    return corrected_color_mask

def disc_to_vert_and_class(highest_disc, condition):
    n = 15 if condition == "spinal_canal_stenosis" else 0
    if highest_disc == "l1_l2": return "l1", (32 - n)
    elif highest_disc == "l2_l3": return "l2", (33 - n)
    elif highest_disc == "l3_l4": return "l3", (34 - n)
    elif highest_disc == "l4_l5": return "l4", (35 - n)
    elif highest_disc == "l5_s1": return "l5", (36 - n)


def align_verts_to_val(highest_vert, vert_val):
    if highest_vert == "l1": 
        return {"l1": vert_val + 0, "l2": vert_val + 1, "l3": vert_val + 2, "l4": vert_val + 3, "l5": vert_val + 4, "s1": vert_val + 5}
    elif highest_vert == "l2": 
        return {"l1": vert_val - 1, "l2": vert_val + 0, "l3": vert_val + 1, "l4": vert_val + 2, "l5": vert_val + 3, "s1": vert_val + 4}
    elif highest_vert == "l3": 
        return {"l1": vert_val - 2, "l2": vert_val - 1, "l3": vert_val + 0, "l4": vert_val + 1, "l5": vert_val + 2, "s1": vert_val + 3}
    elif highest_vert == "l4": 
        return {"l1": vert_val - 3, "l2": vert_val - 2, "l3": vert_val - 1, "l4": vert_val + 0, "l5": vert_val + 1, "s1": vert_val + 2}
    elif highest_vert == "l5": 
        return {"l1": vert_val - 4, "l2": vert_val - 3, "l3": vert_val - 2, "l4": vert_val - 1, "l5": vert_val + 0, "s1": vert_val + 1} 
    else:
        print(f"Vertebrae {highest_vert} invalid!")
    #There is no "s1" condition.
        
import matplotlib.patches as patches

def find_vert_with_coords(mask, x, y):
    central_x_est, central_y_est = int(x) - 11, int(y) - 8
    vert_val = mask[central_y_est, central_x_est]
    if vert_val == 0:
        offsets = [(dx, dy) for dx in range(-3, 4) for dy in range(-3, 4)]
        for dx, dy in offsets:
            temp_x_est, temp_y_est = central_x_est + dx, central_y_est + dy
            
            if 0 <= temp_y_est < mask.shape[0] and 0 <= temp_x_est < mask.shape[1]:
                vert_val = mask[temp_y_est, temp_x_est]
                if vert_val != 0:
                    return vert_val
    
        '''#Debugging code.
        fig, ax = plt.subplots()
        ax.imshow(mask, cmap='gray')
        rect = patches.Rectangle((central_x_est - 3, central_y_est - 3), 7, 7, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)'''
        return None
    else:
        return vert_val

def locate_vertebrae(mask, highest_vert_str, highest_vert_class, x, y, all_vertebrae = ["l1", "l2", "l3", "l4", "l5", "s1"]):
    vertebrae_val = {v: 0 for v in all_vertebrae}
    #Find the highest vertebrae to act as a reference point within the image, as to what vertebrae are present and their class number.
    vert_val = find_vert_with_coords(mask, x, y)
    if vert_val == None:
        return None
    #Align this with order now.
    vert_val_dict = align_verts_to_val(highest_vert_str, vert_val)
    return vert_val_dict

def calculate_disc_bboxes(regions):
    #Unpack lx and lx+1 vert bboxes.
    bboxes = []
    for i in range(len(regions) - 1):
        top_u, left_u, bottom_u, right_u = regions[i] #Box above.
        top_l, left_l, bottom_l, right_l = regions[i + 1] #Box below.
        #Choose largest out of the x values, so nothing is missed.
        bbox = bottom_u - 3, min(left_u, left_l), top_l + 3, max(right_u + 5, right_l + 5) #Add some height to the bbox to fill dead space created otherwise.
        bboxes.append(bbox)
    return bboxes

def label_sag_mask(path, mask, chunk, classes, class_numbers_verts):    
    chunk = chunk.sort_values("level").reset_index() #Sort down l1_l2 -> l5_s1.
    x, y, level = chunk.x[0], chunk.y[0], chunk.level
    highest_disc = level[0]
    condition = chunk["condition"].values[0]
    highest_vert_str, highest_vert_class = disc_to_vert_and_class(highest_disc, condition)

    vert_val_dict = locate_vertebrae(mask, highest_vert_str, highest_vert_class, x, y) #Use these values to switch the values on the color mask to their respective class values.
    if vert_val_dict is None:
        #print("vert_val_dict is None.")
        return None
    for idx, vert_val in enumerate(vert_val_dict.values()):
        mask[mask == vert_val] = class_numbers_verts[idx] #Won't line up due to them being about 20-30 difference.
    #Assign all values but class_numbers and 0 to be a 1 (other_disc class).

    class_mask = mask.copy().astype(int)
    acceptable_vals = class_numbers_verts + [0]
    filter_mask = ~np.isin(class_mask, acceptable_vals) #Makes all other detected islands of pixels as a default value.
    class_mask[filter_mask] = 1 #~[0, 1, 32...37]

    no_other_mask = class_mask.copy()
    no_other_mask[no_other_mask == 1] = 0 #Make a copy of the classly labeled mask and remove the other_discs to simplify finding the disc regions between the vertebrae.
    no_other_mask = no_other_mask.astype(int)
    debug = np.unique(no_other_mask.copy())
    no_verts = len(np.unique(no_other_mask)) - 1
    regions = []
    for i in np.unique(no_other_mask):
        if i == 0:
            continue
        temp_mask = np.where(no_other_mask == i, i, 0)
        region = regionprops(temp_mask)
        regions.append(region[0].bbox)
    disc_bboxes = calculate_disc_bboxes(regions)

    #Using the class numbers and the bounding boxes, fill the class mask.
    class_numbers = chunk["class_number"].to_list() #Will be the values to assign the discs to (top down).
    def_discs = [2, 3, 4, 5, 6]
    if "right" in condition:
        def_discs = list(np.array(def_discs) + 15)

    all_discs = {"l1_l2": 0, "l2_l3": 0, "l3_l4": 0, "l4_l5": 0, "l5_s1": 0}
    for idx, disc in enumerate(all_discs.keys()):
        disc_info = chunk[chunk["level"] == disc]
        if disc_info.empty:
            all_discs[disc] = def_discs[idx]
        else:
            all_discs[disc] = class_numbers[0]
            class_numbers.remove(class_numbers[0])

    passive_mask = np.zeros_like(class_mask)
    class_numbers_padded = list(all_discs.values())
    for idx, bbox in enumerate(disc_bboxes):
        bottom, left, top, right = bbox
        passive_mask[bottom:top, left:right] = class_numbers_padded[idx]
    merged_mask = np.where(class_mask > 0, class_mask, passive_mask)                                #no_discs = no_vertebrae - 1.
    if 2 * no_verts - 3 != len(np.unique(merged_mask)) - 2: #Must follow the equation no_vertebrae + no_discs - classes[0, 1] == unique - classes[0, 1].
        return merged_mask
    else:
        return None


def save_mask_as_png(mask, fname):
    assert mask.shape == (224, 224)
    fname = fname.replace("nnUNet_segments", "nnUNet_segments_labeled").replace("npy", "png")
    save_folder = "/".join(fname.split('/')[:-1])
    os.makedirs(save_folder, exist_ok=True)
    mask = mask.astype(np.uint8)
    mask_pil = Image.fromarray(mask)
    mask_pil.save(fname)
    

In [33]:
fail_rates = {"Successful": 0, "Fail to load": 0, "Fail within loop": 0}
print(f"Labeling {len(st1_class_df.groupby(["series_id", "instance_number"]))} Sagittal T1 MRIs.")

for idx, chunk in st1_class_df.groupby(["series_id", "instance_number"]):
    segment_path = os.path.join("nnUNet_segments", str(chunk["study_id"].values[0]),
                                 str(chunk["series_id"].values[0]), f"{chunk["instance_number"].values[0]}.npy") 
    if os.path.isfile(segment_path):
        gray_mask = np.load(segment_path) #Is grayscale img.
    else:
        fail_rates["Fail to load"] = fail_rates["Fail to load"] + 1
        continue
    color_mask = auto_label_and_correct_mask(gray_mask, st1_classes)
    png_path = segment_path.replace("npy", "png").replace("nnUNet_segments", "train_png")
    sag_t1_mask = label_sag_mask(path = png_path, mask = color_mask, chunk = chunk, classes = st1_classes, 
                                    class_numbers_verts = [32, 33, 34, 35, 36, 37])
    if sag_t1_mask is not None:
        fail_rates["Successful"] = fail_rates["Successful"] + 1
        save_mask_as_png(sag_t1_mask, segment_path)
    else:
        fail_rates["Fail within loop"] = fail_rates["Fail within loop"] + 1
print(fail_rates)

Labeling 8282 Sagittal T1 MRIs.
{'Successful': 2980, 'Fail to load': 1852, 'Fail within loop': 3450}


In [34]:
fail_rates = {"Successful": 0, "Fail to load": 0, "Fail within loop": 0}
print(f"Labeling {len(st2_class_df.groupby(["series_id", "instance_number"]))} Sagittal T2 MRIs.")
for idx, chunk in st2_class_df.groupby(["series_id", "instance_number"]):
    segment_path = os.path.join("nnUNet_segments", str(chunk["study_id"].values[0]),
                                str(chunk["series_id"].values[0]), f"{chunk["instance_number"].values[0]}.npy")
    if os.path.isfile(segment_path):
        gray_mask = np.load(segment_path)
    else:
        fail_rates["Fail to load"] = fail_rates["Fail to load"] + 1
        continue
    color_mask = auto_label_and_correct_mask(gray_mask, st2_classes)
    png_path = segment_path.replace("npy", "png").replace("nnUNet_segments", "train_png")
    sag_t2_mask = label_sag_mask(path = png_path, mask = color_mask, chunk = chunk, classes = st2_classes, 
                                    class_numbers_verts = [17, 18, 19, 20, 21, 22])
    if sag_t2_mask is not None:
        fail_rates["Successful"] = fail_rates["Successful"] + 1
        save_mask_as_png(sag_t2_mask, segment_path)
    else:
        fail_rates["Fail within loop"] = fail_rates["Fail within loop"] + 1
print(fail_rates)
print(f"Total number of Sagittal Samples: {len(glob(os.path.join("nnUNet_segments_labeled", "*/*/*png")))}")

Labeling 2521 Sagittal T2 MRIs.
{'Successful': 1869, 'Fail to load': 115, 'Fail within loop': 537}
Total number of Sagittal Samples: 4849


In [35]:
def path_to_info(path):
    """Takes a path in the style that RSNA use."""
    sp = path.split('/')
    return sp[-3], sp[-2], sp[-1].split('.')[0] #Study_id, series_id, instance_number.

In [36]:
"""2.5D approach"""
#First check the number of 2 adjacent slices within the data.
t1_pairs, t1_triplets, t2_pairs, t2_triplets = [], [], [], []
for study_id in os.listdir("nnUNet_segments_labeled"):
    folder_path = os.path.join("nnUNet_segments_labeled", study_id)
    for series_id in os.listdir(folder_path):
        #Sagittal T1 or Sagittal T2/STIR
        sd = train_series_descriptions[train_series_descriptions["series_id"] == int(series_id)]["series_description"].values[0]
        series_path = os.path.join(folder_path, series_id)
        filenames = os.listdir(series_path)
        filenames = [int(filename.removesuffix('.png')) for filename in filenames]
        for instance in filenames:
            if (instance + 1) in filenames and (instance + 2) in filenames:
                if sd == "Sagittal T1":
                    t1_triplets.extend([os.path.join(series_path, f"{instance}.png"), os.path.join(series_path, f"{instance + 1}.png")])
                elif sd == "Sagittal T2/STIR":
                    t2_triplets.extend([os.path.join(series_path, f"{instance}.png"), os.path.join(series_path, f"{instance + 1}.png")])
            elif (instance + 1) in filenames:
                if sd == "Sagittal T1":
                    t1_pairs.extend([os.path.join(series_path, f"{instance}.png"), os.path.join(series_path, f"{instance + 1}.png")])
                elif sd == "Sagittal T2/STIR":
                    t2_pairs.extend([os.path.join(series_path, f"{instance}.png"), os.path.join(series_path, f"{instance + 1}.png")])

print(f"ST1 Pairs: {len(t1_pairs) // 2} \nST2 Pairs: {len(t2_pairs) // 2}") #As we can see, there are enough pairs but not enought triplets.
print(f"ST1 Triplets: {len(t1_triplets) // 3} \nST2 Triplets: {len(t2_triplets) // 3}")

ST1 Pairs: 583 
ST2 Pairs: 253
ST1 Triplets: 54 
ST2 Triplets: 19


In [84]:
"""Stack masks and imgs on each other."""
import nibabel as nib
STACKED_ST1_MASKS_DIR = "nnUNet_raw/Dataset001_ST1_Degeneration/labelsTr"
STACKED_ST1_IMGS_DIR = "nnUNet_raw/Dataset001_ST1_Degeneration/imagesTr"
STACKED_ST2_MASKS_DIR = "nnUNet_raw/Dataset002_ST2_Degeneration/labelsTr"
STACKED_ST2_IMGS_DIR = "nnUNet_raw/Dataset002_ST2_Degeneration/imagesTr"
os.makedirs(STACKED_ST1_MASKS_DIR, exist_ok=True)
os.makedirs(STACKED_ST1_IMGS_DIR, exist_ok=True)
os.makedirs(STACKED_ST2_MASKS_DIR, exist_ok=True)
os.makedirs(STACKED_ST2_IMGS_DIR, exist_ok=True)

def stack_arrays(img1, img2):
    arr1 = np.array(img1).astype(np.uint8)
    arr2 = np.array(img2).astype(np.uint8)
    arrd = np.zeros_like(img1).astype(np.uint8)
    arr_stacked = np.stack([arr1, arr2, arrd], axis = 1)
    return arr_stacked.transpose(0, 2, 1)

def save_as_nii(arr, filepath):
    nii_image = nib.Nifti1Image(arr, affine=np.eye(4)) #Default affine.
    nib.save(nii_image, filepath)   

def save_as_png(img, filepath):
    #img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255
    img = img.astype(np.uint8)
    img = Image.fromarray(img)
    img.save(filepath) 

def process_pair(mask_path1, mask_path2, stacked_mask_dir, stacked_imgs_dir, file_start, idx):
    img_path1, img_path2 = mask_path1.replace('nnUNet_segments_labeled', 'train_png'), mask_path2.replace('nnUNet_segments_labeled', 'train_png')

    mask1, mask2 = Image.open(mask_path1).convert('L'), Image.open(mask_path2).convert('L')
    img1, img2 = Image.open(img_path1).convert('L'), Image.open(img_path2).convert('L')

    stacked_mask = stack_arrays(mask1, mask2)
    stacked_img = stack_arrays(img1, img2)

    new_mask_path = os.path.join(stacked_mask_dir, f"{file_start}_{f'{idx:04d}'}.png")
    new_img_path = os.path.join(stacked_imgs_dir, f"{file_start}_{f'{idx:04d}'}_0000.png")

    save_as_png(stacked_mask, new_mask_path)
    save_as_png(stacked_img, new_img_path)

for i in range(len(t1_pairs) // 2):
    back_mask_path, front_mask_path = t1_pairs[2 * i], t1_pairs[2 * i + 1]
    process_pair(back_mask_path, front_mask_path, STACKED_ST1_MASKS_DIR, STACKED_ST1_IMGS_DIR, "ST1", idx = i)

for i in range(len(t2_pairs) // 2):
    back_mask_path, front_mask_path = t2_pairs[2 * i], t2_pairs[2 * i + 1]
    process_pair(back_mask_path, front_mask_path, STACKED_ST2_MASKS_DIR, STACKED_ST2_IMGS_DIR, "ST2", idx = i)

In [85]:
"""Change filenames of train imgs and masks then move to the appropriate folder."""
def sd_to_bin(sd):
    if sd == "Sagittal T1": return "0000"
    if sd == "Sagittal T2/STIR": return "0001"
    if sd == "Axial T2": return "0002"

In [86]:
"""Preprocess test images and rename to format. -> seriesID_instanceNumber_000(0,1,2).png"""
test_img_dir = "rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939"
save_folders = ["nnUNet_raw/Dataset001_ST1_Degeneration/imagesTs", "axial_test_png", "nnUNet_raw/Dataset002_ST2_Degeneration/imagesTs"]
for idx, dicom_folder in enumerate(os.listdir(test_img_dir)):
    sd = test_series_description[test_series_description["series_id"] == int(dicom_folder.split('/')[-1])]
    sd = sd["series_description"].values[0]
    bin_sd = sd_to_bin(sd)
    plane = "sagittal" if sd != "Axial T2" else "axial"
    dicom_data_3d = load_dicom_stack(os.path.join(test_img_dir, dicom_folder), plane = plane)
            
    if dicom_data_3d["array"].shape[0] == 0:
        print(f"Dicom data size 0: {dicom_data_3d["array"].shape}")
        continue
            
    save_folder = save_folders[idx]
    os.makedirs(save_folder, exist_ok = True)
    #Using 2.5D setup, it is probably best to use 2.5D prediction, this way the model has more context, also that the model sees each important img twice.
    for k in range(dicom_data_3d["array"].shape[0] - 1): 
        img1 = dicom_data_3d["array"][k]
        img2 = dicom_data_3d["array"][k + 1]
        pixel_spacing = dicom_data_3d["pixel_spacing"]
        processed_img1 = preprocess_img(img1, pixel_spacing)
        processed_img2 = preprocess_img(img2, pixel_spacing)
        nn_img_path = os.path.join(save_folder, f"ST1_{k:04d}_0000.png") #{bin_to_sd}
        img_stacked = np.stack([processed_img1, processed_img2, np.zeros_like(processed_img1)], axis = 2)
        save_as_png(img_stacked, nn_img_path)
       

In [98]:
st1_dataset = {
    "name": "Sagittal T1 Degeneration",
    "description": "Using Sagittal T1 images, identify areas of right/left neural foraminal narrowing",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "tensorImageSize": "2D",

    "channel_names": { #Input channels into the network.
        "0": "slice_n", 
        "1": "slice_n+1",
        "2": "zeros_like",
    }, 
    "labels": st1_classes,
    
    "numTraining": len(glob(os.path.join(UNET_ST1_DIR, "imagesTr", "*png"))), 
    "file_ending": ".png"
    }
with open(os.path.join(UNET_ST1_DIR, 'dataset.json'), 'w') as json_file:
    json.dump(st1_dataset, json_file, indent=4)

st2_dataset = {
    "name": "Sagittal T2/STIR Degeneration",
    "description": "Using Sagittal T2/STIR images, identify areas of central spinal canal stenosis",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "tensorImageSize": "2D",

    "channel_names": { #Input channels into the network.
        "0": "slice_n", 
        "1": "slice_n+1",
        "2": "zeros_like",
    }, 
    "labels": st2_classes,
    
    "numTraining": len(glob(os.path.join(UNET_ST2_DIR, "imagesTr", "*png"))), 
    "file_ending": ".png"
    }
with open(os.path.join(UNET_ST2_DIR, 'dataset.json'), 'w') as json_file:
    json.dump(st2_dataset, json_file, indent=4)

In [99]:
""" #I CAN ACTUALLY JUST PUT ALL OF THIS IN ONE DATASET BUT KEEP SEPARATE FOR NOW DUE TO CONVENIENCE.
#Install nnUNet libraries.
!pip install nnunetv2

#Setup folder directories.
export nnUNet_preprocessed="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_preprocessed"
export nnUNet_results="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results"
export nnUNet_raw="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw"

#Plan and preprocess Sagittal T1 and T2.
nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity
nnUNetv2_plan_and_preprocess -d 002 --verify_dataset_integrity
"""


' #I CAN ACTUALLY JUST PUT ALL OF THIS IN ONE DATASET BUT KEEP SEPARATE FOR NOW DUE TO CONVENIENCE.\n#Install nnUNet libraries.\n!pip install nnunetv2\n\n#Setup folder directories.\nexport nnUNet_preprocessed="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_preprocessed"\nexport nnUNet_results="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results"\nexport nnUNet_raw="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw"\n\n#Plan and preprocess Sagittal T1 and T2.\nnnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity\nnnUNetv2_plan_and_preprocess -d 002 --verify_dataset_integrity\n'

In [100]:
"""
#Train and Predict Dataset 001.       (Best dice score found at ~50 epochs)
nnUNetv2_train 001 2d 0 -device mps 
nnUNetv2_predict -i "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw/Dataset001_ST1_Degeneration/imagesTs" -o "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results/output" -d 001 -c 2d -f 0 --save_probabilities -device mps

#Train and Predict Dataset 002.
nnUNetv2_train 002 2d 0 -device mps 
nnUNetv2_predict -i "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw/Dataset002_ST2_Degeneration/imagesTs" -o "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results/output" -d 002 -c 2d -f 0 --save_probabilities -device mps
"""


'\n#Train and Predict Dataset 001.       (Best dice score found at ~50 epochs)\nnnUNetv2_train 001 2d 0 -device mps \nnnUNetv2_predict -i "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw/Dataset001_ST1_Degeneration/imagesTs" -o "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results/output" -d 001 -c 2d -f 0 --save_probabilities -device mps\n\n#Train and Predict Dataset 002.\nnnUNetv2_train 002 2d 0 -device mps \nnnUNetv2_predict -i "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw/Dataset002_ST2_Degeneration/imagesTs" -o "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results/output" -d 002 -c 2d -f 0 --save_probabilities -device mps\n'

In [101]:
"""Visualize predictions on the test data."""
ST1_PREDICTIONS_OUTPUT = "nnUNet_results/output"
ST2_PREDICTIONS_OUTPUT = "nnUNet_results/output"
for idx, path in enumerate(list(sorted(glob(os.path.join(ST2_PREDICTIONS_OUTPUT, "*png"))))):
    data = nib.load(path)
    test_path = path.replace("nnUNet_results", "nnUNet_raw/Dataset002_ST2_Degeneration").replace("output", "imagesTs")
    f, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(nib.load(test_path))
    ax2.imshow(data)
    plt.show()

    #logits_mask = data["probabilities"] #This contains a (class_number, 1, 224, 224) of (raw?) logits.
    #print(f"Prediction {idx}:\nshape: {logits_mask.shape}\nmax value: {np.max(logits_mask)}\nmin value: {np.min(logits_mask)}")

In [102]:
"""Work out the probabilites using the logit masks."""
sag_t1_groups = {
    "left_neural_foraminal_narrowing_l1_l2": {2: 0, 7: 0, 12: 0}, #Condition and their class_numbers.
    "left_neural_foraminal_narrowing_l2_l3": {3: 0, 8: 0, 13: 0},
    "left_neural_foraminal_narrowing_l3_l4": {4: 0, 9: 0, 14: 0},
    "left_neural_foraminal_narrowing_l4_l5": {5: 0, 10: 0, 15: 0},
    "left_neural_foraminal_narrowing_l5_s1": {6: 0, 11: 0, 16: 0},

    "right_neural_foraminal_narrowing_l1_l2": {17: 0, 22: 0, 27: 0},
    "right_neural_foraminal_narrowing_l2_l3": {18: 0, 23: 0, 28: 0},
    "right_neural_foraminal_narrowing_l3_l4": {19: 0, 24: 0, 29: 0},
    "right_neural_foraminal_narrowing_l4_l5": {20: 0, 25: 0, 30: 0},
    "right_neural_foraminal_narrowing_l5_s1": {21: 0, 26: 0, 31: 0},
}

sag_t2_groups = {
    "spinal_canal_stenosis_l1_l2": {2: 0, 7: 0, 12: 0}, #Condition and their class_numbers.
    "spinal_canal_stenosis_l2_l3": {3: 0, 8: 0, 13: 0},
    "spinal_canal_stenosis_l3_l4": {4: 0, 9: 0, 14: 0},
    "spinal_canal_stenosis_l4_l5": {5: 0, 10: 0, 15: 0},
    "spinal_canal_stenosis_l5_s1": {6: 0, 11: 0, 16: 0},
}

In [103]:
def process_logits(raw_logits, s, condition):
    #Drop irrelevant logit (0th) dimensions.
    #print(f"raw logits shape {raw_logits.shape}")
    relevant_logits = raw_logits[s].reshape(5, 1, 3, 224, 224)
    #print(f"relevant logits shape: {relevant_logits.shape}")
    
    summed_mask_faces = np.sum(relevant_logits, axis = (-1, -2))
    print(f"summed_mask_faces shape: {summed_mask_faces.shape}")

    if condition == "narrowing":
        summed_mask_faces = summed_mask_faces.reshape(5, 2, 3)
        #Determine right or left by the sum of the logits in each 1st dimensional slice. (Left is slice [:, 0])
        lmask = np.sum(summed_mask_faces[:, 0], axis = 1) 
        rmask = np.sum(summed_mask_faces[:, 1], axis = 1)
        if np.sum(rmask) > np.sum(lmask):
            summed_mask_faces[:, 0] = 0 
        else:
            summed_mask_faces[:, 1] = 0
    else:
        summed_mask_faces.reshape(5, 1, 3)
        
    #Softmax each set of 3 logits.
    probabilities = np.array(F.softmax(torch.tensor(summed_mask_faces), dim = 0))
    print(f"probs shape: {probabilities.shape}")
    print(f"example of SM probs: {probabilities[0]}")
    return probabilities

def vertebrae_to_confidence(logits, mask):
    vertebrae_logits = logits[mask]
    vert_pix_count = np.sum(np.where(vertebrae_logits > 0))
    if vert_pix_count > 5500:
        return 1
    elif vert_pix_count < 5500:
        return vert_pix_count / 5500

In [104]:
"""Patient-wide probabilites, but first -> Unpack the 2.5D predictions into 2 2D images."""
def process_raw_outputs(mask_groups, predictions_output_path, p_mask, vert_mask, p_sum, condition, confidences = []):
    for s in mask_groups.values():
        for path in list(sorted(glob(os.path.join(predictions_output_path, "*npz")))):
            data = np.load(path)
            logits = data["probabilities"] #Shape: (no_classes, 2, 224, 224).
            for dimension in range(logits.shape[1]): #Loop over each of the stacked masks.
                probabilities = process_logits(logits[:, dimension], s, p_mask, condition)
                #mask_confidence = vertebrae_to_confidence(logits[:, dimension], vert_mask) #Remove confidence value for now, to simplify.
                p_sum = p_sum + probabilities # * mask_confidence
                #confidences.append(mask_confidence)
    p_avg = p_sum / len(glob(os.path.join(predictions_output_path, "*npz")))
    return p_avg

#st1_severity = process_raw_outputs(ST1_PREDICTIONS_OUTPUT, p_mask = list(range(2, 31, 1)), vert_mask = list(range(32, 37, 1)))

st2_severity = process_raw_outputs(sag_t2_groups, 
                                   ST2_PREDICTIONS_OUTPUT, 
                                   p_mask = list(range(2, 17, 1)), 
                                   vert_mask = list(range(17, 23, 1)),
                                   p_sum = np.zeros((5, 3)), 
                                   condition = "canal_stenosis")
print(st2_severity)

[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]


In [105]:
################################################################################################################################################
######################################################## AXIAL PREDICTIONS #####################################################################
################################################################################################################################################

In [106]:
"""Convert 2D pngs into 3D"""

'Convert 2D pngs into 3D'

In [107]:
"""Slice the train df and convert the severities into binary arrays."""
rtrain = train[train.columns[16:]].copy()
def sev_to_bin(sev):
    if sev == "Normal/Mild": return [1, 0, 0]
    elif sev == "Moderate": return [0, 1, 0]
    elif sev == "Severe": return [0, 0, 1]

"""def study_to_fpath(st):
    rdf = ax_sds[ax_sds["study_id" == int(st)]]
    si = rdf["series_id"].values[0]
    filepath = os.path.join(str(st), str(si), f"0.npy")"""


for col in rtrain.columns[1:]:
    rtrain[col] = rtrain[col].apply(sev_to_bin)

"""ax_sds = train_series_descriptions[train_series_descriptions["series_description"] == "Axial T2"]
filepaths = rtrain[0].apply(study_to_fpath)"""

'ax_sds = train_series_descriptions[train_series_descriptions["series_description"] == "Axial T2"]\nfilepaths = rtrain[0].apply(study_to_fpath)'

In [108]:
"""Insert severities into renamed sample_submission.csv"""
sub_temp = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv")
submission_columns = sub_temp.columns
#Copy the first column of the submission. Structure -> left_foraminal, left_subarticular, right_for, right_sub, canal_stenosis (25 rows).
col1 = sub_temp["row_id"].to_list()

#Temporary Solution: default axial severities -> [0.33..., 0.33..., 0.33...] x 10 rows.
st1_severity = np.ones((10, 3)) * (1 / 3)
ax_severity = np.ones((10, 3)) * (1 / 3)

zipped_severities = np.array([st1_severity[:5],
                              ax_severity[:5],
                              st1_severity[5:],
                              ax_severity[5:],
                              st2_severity])

submission = pd.DataFrame({"row_id": col1, 
                           "normal_mild": zipped_severities[0].flatten(), 
                           "moderate": zipped_severities[1].flatten(), 
                           "severe": zipped_severities[2].flatten()})


ValueError: All arrays must be of the same length

In [ ]:
"""testing zone. keep clear."""




